# Obtenemos la primera información de toda la región de la Araucania

In [28]:
import requests
import io
from bs4 import BeautifulSoup
from PyPDF2 import PdfFileReader
import pandas as pd

## Obtendremos los enlaces y fechas para cada reporte

In [2]:
URL = 'https://seremi9.redsalud.gob.cl/accion-sanitaria-3/registros-covid19/'
page = requests.get(URL) #Obtenemos la respuesta del servidor

In [3]:
soup     = BeautifulSoup(page.content, 'html.parser')  #Transformamos a un objeto soup

In [4]:
job_elems = soup.find_all('ol')                    # Buscamos todos los elementos "ol"
for job_elem in job_elems:                         # Recorremos
    enlaces = job_elem.find_all('a')               # Buscamos los enlaces de ahí

In [5]:
reportes_diarios = []
base_url         = "https://seremi9.redsalud.gob.cl"
for enlace in enlaces:                            # Recorremos los enlaces
    fecha  = enlace.text                          # obtenemos el texto
    enlace = enlace['href']                       # obtenemos el enlace
    url    = base_url + enlace
    dato   = {'fecha': fecha, 'url':url}
    reportes_diarios.append(dato)

## Desde ahora trabajamos con los reportes diarios

In [104]:
#Esta funcion retornara en un arreglo el texto de la pagina 0 del PDF
def get_info(fecha, url):
    print ("Obteniendo información del: " + fecha)
    page      = requests.get(url)                           #Obtenemos la respuesta del servidor
    f         = io.BytesIO(page.content)                    #Tenemos el pdf
    reader    = PdfFileReader(f)                            #leemos el pf 
    contenido = reader.getPage(0).extractText().split('\n') #Tenemos el pdf separado por salto de linea
    
    
    #Filtramos e iteramos los datos
    pos_ini   = contenido.index('Angol')
    pos_fin   = contenido.index('Total General')
    
    contenido = contenido[pos_ini:pos_fin]                  # Filtramos con los datos que queremos    
    contenido.remove("Chol")
    contenido.remove("-")
    contenido = ["Chol-Chol" if x=="Chol" else x for x in contenido] #Cambiamos el nombre de cholchol
    
    #Eliminamos los espacios en blanco
    contenido_filtrado = []
    for c in contenido:
        if c != " ":
            contenido_filtrado.append(c)
    
    return contenido_filtrado    

In [105]:
# Retornará si el elemento ingresado es una ciudad
def is_city(elemento):
    bandera  = False
    ciudades = ['Angol', 'Carahue', 'Chol-Chol', 'Collipulli', 'Cunco', 'Curacautín', 'Curarrehue', 'Ercilla', 'Freire', 'Galvarino', 'Gorbea', 'Lautaro', 'Loncoche', 'Lonquimay', 'Los Sauces', 'Lumaco',  'Melipeuco', 'Nueva Imperial', 'Padre Las Casas',  'Perquenco', 'Pitrufquén', 'Pucón', 'Renaico',  'Saavedra', 'Temuco', 'Teodoro Schmidt','Toltén', 'Traiguén',  'Victoria', 'Vilcún', 'Villarrica', 'Purén']
    if elemento in ciudades:
        bandera = True
    return bandera

In [116]:
#Retornará los datos acumulados de los casos por region
def get_data(elemento, f):
    print(len(elemento))
    datos = [f]
    print(elemento)
    anterior = '' 
    ciudad   = ''
    n = 0
    for e in elemento:
        if (is_city(e) == False):
            anterior += e
        else:
            if anterior != '':
                #print (ciudad + " : " + anterior)
                datos.append(int(anterior))
                anterior = ''
        
        if is_city(e):
            ciudad = e
        
        ## Eliminamos el error del ultimo elemento
        if n == len(elemento)-1:
            datos.append(int(anterior))
        n += 1    
    #datos.append(elemento[len(elemento)-1]) #Para solucionar el ultimo elemento
    print(elemento[len(elemento)-1])
    return datos

La siguiente traerá los nombre de las ciudades que entrega el reporte diario.

In [107]:
def get_citys(fecha, url):
    ciudades = ['Fecha']
    data     = get_info(fecha, url)
    for d in data:
        if is_city(d):
            ciudades.append(d)
    return ciudades

ciudades = get_citys(reportes_diarios[0]['fecha'], reportes_diarios[0]['url'])

Obteniendo información del: 01 de Mayo


### Crearemos el dataframe de la región

In [117]:
araucania = pd.DataFrame(columns=ciudades)
araucania

,Fecha,Angol,Carahue,Chol-Chol,Collipulli,Cunco,Curacautín,Curarrehue,Ercilla,Freire,...,Purén,Renaico,Saavedra,Temuco,Teodoro Schmidt,Toltén,Traiguén,Victoria,Vilcún,Villarrica


### Llenamos de información el dataframe

In [118]:
for r in reportes_diarios:
    texto     = get_info(r['fecha'], r['url'])       # Obtenemos el texto de la primera pagina del informe
    data      = get_data(texto, r['fecha'])          # Obtenemos la data
    print (data)
    araucania.loc[len(araucania)] = data             # Creamos el data frame

Obteniendo información del: 01 de Mayo
64
['Angol', '102', 'Carahue', '30', 'Chol-Chol', '4', 'Collipulli', '6', 'Cunco', '5', 'Curacautín', '5', 'Curarrehue', '4', 'Ercilla', '9', 'Freire', '7', 'Galvarino', '2', 'Gorbea', '12', 'Lautaro', '39', 'Loncoche', '18', 'Lonquimay', '0', 'Los Sauces', '1', 'Lumaco', '6', 'Melipeuco', '0', 'Nueva Imperial', '43', 'Padre Las Casas', '97', 'Perquenco', '5', 'Pitrufquén', '17', 'Pucón', '14', 'Purén', '4', 'Renaico', '6', 'Saavedra', '25', 'Temuco', '679', 'Teodoro Schmidt', '5', 'Toltén', '8', 'Traiguén', '2', 'Victoria', '75', 'Vilcún', '42', 'Villarrica', '19']
19
['01 de Mayo', 102, 30, 4, 6, 5, 5, 4, 9, 7, 2, 12, 39, 18, 0, 1, 6, 0, 43, 97, 5, 17, 14, 4, 6, 25, 679, 5, 8, 2, 75, 42, 19]
Obteniendo información del: 02 de Mayo
64
['Angol', '106', 'Carahue', '30', 'Chol-Chol', '4', 'Collipulli', '6', 'Cunco', '5', 'Curacautín', '6', 'Curarrehue', '4', 'Ercilla', '9', 'Freire', '7', 'Galvarino', '2', 'Gorbea', '12', 'Lautaro', '39', 'Loncoche',

71
['Angol', '1', '20', 'Carahue', '3', '5', 'Chol-Chol', '4', 'Collipulli', '6', 'Cunco', '6', 'Curacautín', '7', 'Curarrehue', '4', 'Ercilla', '11', 'Freire', '12', 'Galvarino', '3', 'Gorbea', '13', 'Lautaro', '5', '4', 'Loncoche', '20', 'Lonquimay', '11', 'Los Sauces', '1', 'Lumaco', '7', 'Melipeuco', '0', 'Nueva Imperial', '4', '8', 'Padre Las Casas', '102', 'Perquenco', '5', 'Pitrufquén', '17', 'Pucón', '16', 'Purén', '7', 'Renaico', '12', 'Saavedra', '5', '3', 'Temuco', '73', '6', 'Teodoro Schmidt', '5', 'Toltén', '8', 'Traiguén', '2', 'Victoria', '7', '9', 'Vilcún', '50', 'Villarrica', '20']
20
['12 de Mayo', 120, 35, 4, 6, 6, 7, 4, 11, 12, 3, 13, 54, 20, 11, 1, 7, 0, 48, 102, 5, 17, 16, 7, 12, 53, 736, 5, 8, 2, 79, 50, 20]
Obteniendo información del: 13 de Mayo
74
['Angol', '1', '20', 'Carahue', '3', '5', 'Chol-Chol', '5', 'Collipulli', '6', 'Cunco', '6', 'Curacautín', '7', 'Curarrehue', '4', 'Ercilla', '11', 'Freire', '1', '3', 'Galvarino', '3', 'Gorbea', '13', 'Lautaro', '5',

75
['Angol', '1', '2', '9', 'Carahue', '42', 'Chol-Chol', '6', 'Collipulli', '13', 'Cunco', '21', 'Curacautín', '8', 'Curarrehue', '4', 'Ercilla', '11', 'Freire', '31', 'Galvarino', '3', 'Gorbea', '1', '7', 'Lautaro', '5', '7', 'Loncoche', '2', '1', 'Lonquimay', '56', 'Los Sauces', '1', 'Lumaco', '8', 'Melipeuco', '0', 'Nueva Imperial', '5', '1', 'Padre Las Casas', '1', '28', 'Perquenco', '5', 'Pitrufquén', '2', '4', 'Pucón', '21', 'Purén', '6', 'Renaico', '21', 'Saavedra', '80', 'Temuco', '800', 'Teodoro Schmidt', '6', 'Toltén', '12', 'Traiguén', '9', 'Victoria', '8', '0', 'Vilcún', '6', '3', 'Villarrica', '2', '2']
2
['23 de Mayo', 129, 42, 6, 13, 21, 8, 4, 11, 31, 3, 17, 57, 21, 56, 1, 8, 0, 51, 128, 5, 24, 21, 6, 21, 80, 800, 6, 12, 9, 80, 63, 22]
Obteniendo información del: 24 de Mayo
73
['Angol', '1', '35', 'Carahue', '43', 'Chol-Chol', '6', 'Collipulli', '14', 'Cunco', '23', 'Curacautín', '7', 'Curarrehue', '4', 'Ercilla', '11', 'Freire', '34', 'Galvarino', '3', 'Gorbea', '1', '

81
['Angol', '1', '62', 'Carahue', '5', '4', 'Chol-Chol', '8', 'Collipulli', '2', '9', 'Cunco', '30', 'Curacautín', '8', 'Curarrehue', '4', 'Ercilla', '1', '6', 'Freire', '70', 'Galvarino', '25', 'Gorbea', '20', 'Lautaro', '81', 'Loncoche', '4', '5', 'Lonquimay', '6', '2', 'Los Sauces', '2', 'Lumaco', '11', 'Melipeuco', '6', 'Nueva Imperial', '5', '4', 'Padre Las Casas', '1', '5', '8', 'Perquenco', '6', 'Pitrufquén', '4', '9', 'Pucón', '42', 'Purén', '8', 'Renaico', '2', '5', 'Saavedra', '8', '7', 'Temuco', '8', '35', 'Teodoro Schmidt', '10', 'Toltén', '1', '6', 'Traiguén', '12', 'Victoria', '9', '3', 'Vilcún', '8', '9', 'Villarrica', '3', '3']
3
['03 de Junio', 162, 54, 8, 29, 30, 8, 4, 16, 70, 25, 20, 81, 45, 62, 2, 11, 6, 54, 158, 6, 49, 42, 8, 25, 87, 835, 10, 16, 12, 93, 89, 33]


In [119]:
araucania

,Fecha,Angol,Carahue,Chol-Chol,Collipulli,Cunco,Curacautín,Curarrehue,Ercilla,Freire,...,Purén,Renaico,Saavedra,Temuco,Teodoro Schmidt,Toltén,Traiguén,Victoria,Vilcún,Villarrica
0,01 de Mayo,102,30,4,6,5,5,4,9,7,...,4,6,25,679,5,8,2,75,42,19
1,02 de Mayo,106,30,4,6,5,6,4,9,7,...,4,7,31,684,5,8,2,75,42,19
2,03 de Mayo,107,34,4,6,5,6,4,9,7,...,5,7,34,687,5,8,2,77,43,19
3,04 de Mayo,108,34,4,6,5,6,4,11,7,...,5,7,34,687,5,8,2,75,43,19
4,05 de Mayo,110,34,4,6,5,6,4,11,7,...,6,9,38,693,5,8,2,76,43,19
5,06 de Mayo,111,34,4,6,5,6,4,11,7,...,6,9,38,696,5,8,2,76,46,20
6,07 de Mayo,113,35,4,6,5,6,4,11,7,...,6,10,38,703,5,8,2,77,47,20
7,08 de Mayo,113,35,4,6,6,7,4,11,7,...,6,12,42,711,5,8,2,78,47,20
8,09 de Mayo,115,35,4,6,6,7,4,11,7,...,7,12,45,715,5,8,2,78,49,20
9,10 de Mayo,117,35,4,6,6,7,4,11,9,...,7,12,50,731,5,8,2,78,49,20
